# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks, code quality and readability (no single-letter variables, PEP8 compliant) etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [1]:
import findspark

findspark.init()

import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, LongType, ShortType

sc = SparkContext(appName="HDFS Homework")
spark = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-10-10 15:27:28,157 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


### Spark SQL Solution

In [37]:
df = spark.read.csv('hdfs:/data/clickstream.csv', header=True, sep='\t', inferSchema=True)
df.registerTempTable('clickstream')
df.limit(3).show()

+-------+----------+----------+----------+----------+
|user_id|session_id|event_type|event_page| timestamp|
+-------+----------+----------+----------+----------+
|    562|       507|      page|      main|1695584127|
|    562|       507|     event|      main|1695584134|
|    562|       507|     event|      main|1695584144|
+-------+----------+----------+----------+----------+



In [64]:
spark.sql(
    '''
    SELECT route, COUNT(*) AS count
    FROM (SELECT concat_ws('-', sort_array(collect_list(struct(timestamp, event_page))).event_page) as route
          FROM clickstream
          LEFT JOIN (SELECT user_id, session_id, MIN(timestamp) as error_ts
                     FROM clickstream
                     WHERE event_type LIKE '%error%'
                     GROUP BY user_id, session_id) AS error_ts_table
          ON clickstream.user_id = error_ts_table.user_id AND clickstream.session_id = error_ts_table.session_id
          WHERE (event_type = 'page') AND (error_ts IS NULL OR timestamp < error_ts)
          GROUP BY clickstream.user_id, clickstream.session_id)
    GROUP BY route
    ORDER BY count DESC
    '''
).show(10)

+-------------------+-----+
|              route|count|
+-------------------+-----+
|               main| 8090|
|       main-archive| 1095|
|        main-rabota| 1039|
|      main-internet|  879|
|         main-bonus|  865|
|          main-news|  760|
|       main-tariffs|  669|
|        main-online|  584|
|         main-vklad|  514|
|main-archive-rabota|  167|
+-------------------+-----+
only showing top 10 rows



### Spark RDD Solution

In [68]:
rdd = sc.textFile("hdfs:/data/clickstream.csv")

In [70]:
rdd1 = rdd.map(lambda row: row.split('\t'))
header = rdd1.first()
rdd2 = rdd1.filter(lambda row: row != header)
rdd3 = rdd2.filter(lambda row: 'error' in row[2])
rdd4 = rdd3.map(lambda row: (row[:2], row[2:]))
rdd4.take(5)

[(['562', '507'], ['wNaxLlerrorU', 'main', '1695584154']),
 (['4567', '514'], ['mAXExoCXerror', 'main', '1695584351']),
 (['461', '174'], ['uvjferrorYYwYlubX', 'internet', '1695584529']),
 (['844', '258'], ['kfIpzqTUaerrorSQD', 'main', '1695584652']),
 (['461', '174'], ['iVerrornrA', 'news', '1695584698'])]

### Spark DF Solution

In [ ]:
# read csv

df = spark.read.csv('hdfs:/data/clickstream.csv', header=True, sep='\t', inferSchema=True)
df.show(5)

In [ ]:
error_ts_df = (df.filter(df.event_type.contains('error'))
                 .groupBy('user_id', 'session_id')
                 .agg({'timestamp': 'min'})
                 .withColumnRenamed('min(timestamp)', 'error_ts'))

In [ ]:
error_ts_df.filter(error_ts_df['session_id'] == 12).show()

In [ ]:
df_join = df.join(error_ts_df, on=['user_id', 'session_id'], how='left')

In [ ]:
(df_join.filter((F.col('event_type') == 'page') & ((F.col('error_ts').isNull()) | (F.col('timestamp') < F.col('error_ts'))))
        .orderBy('user_id', 'session_id', 'timestamp')
        .groupBy('user_id', 'session_id')
        .agg(F.concat_ws('-', F.collect_list(F.col('event_page'))))
        .withColumnRenamed('concat_ws(-, collect_list(event_page))', 'route')
        .groupBy('route')
        .count()
        .sort(F.col('count').desc())
        .show(50))